In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [3]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [4]:
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [5]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [6]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [7]:
t_df=toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
t_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [8]:
toronto_grouped = t_df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_grouped

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [9]:
for index, row in toronto_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_grouped

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [10]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_grouped[toronto_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,Parkview Hill / Woodbine Gardens
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,Wexford / Maryvale
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


In [11]:
toronto_grouped.shape

(103, 3)

In [12]:
coordinates = pd.read_csv("https://cocl.us/Geospatial_data")

coordinates.rename(columns={"Postal Code": "PostalCode"},inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
toronto_grouped2=toronto_grouped.merge(coordinates,on='PostalCode',how='left')
toronto_grouped2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test2_df = pd.DataFrame(columns=column_names)

test2_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test2_list:
    test2_df = test2_df.append(toronto_grouped2[toronto_grouped2["PostalCode"]==postcode], ignore_index=True)
    
test2_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,Wexford / Maryvale,43.750072,-79.295849
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420


In [15]:
address = 'Toronto'
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [16]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto_grouped2['Latitude'], toronto_grouped2['Longitude'], toronto_grouped2['Borough'], toronto_grouped2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3386cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
CLIENT_ID = 'RTBEK5FGAJBNSJIZQZAYI4RNHUR4JPO35M5TYBNIPOUNWJ0I' # your Foursquare ID
CLIENT_SECRET = '2CXRZYT4SCGOEFDHNRKYJV5GLIH2KSMA3CGYYIPX3YL4IGKI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version




In [18]:
toronto_grouped2.loc[45,'Neighborhood']

'Davisville North'

In [19]:
neighborhood_latitude = toronto_grouped2.loc[45, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_grouped2.loc[45, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_grouped2.loc[45, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Davisville North are 43.7127511, -79.3901975.


In [20]:
LIMIT=85
radius=555
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [21]:
results = requests.get(url).json()


In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
import json

venues = results['response']['groups'][0]['items']
    
nearby_venues =pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Sherwood Park,Park,43.716551,-79.387776
1,Summerhill Market North,Food & Drink Shop,43.715499,-79.392881
2,Homeway Restaurant & Brunch,Breakfast Spot,43.712641,-79.391557
3,Hokkaido Sushi,Sushi Restaurant,43.708082,-79.389995
4,Granite Brewery,Brewery,43.707991,-79.389943
5,Winners,Department Store,43.713236,-79.393873
6,Marcheleo's Gourmet Marketplace,Gourmet Shop,43.708041,-79.392195
7,Best Western Roehampton Hotel & Suites,Hotel,43.708878,-79.390880
8,Shoppers Drug Mart,Pharmacy,43.707806,-79.389893
9,Subway,Sandwich Place,43.708474,-79.390674


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

13 venues were returned by Foursquare.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=toronto_grouped2['Neighborhood'],
                                   latitudes=toronto_grouped2['Latitude'],
                                   longitudes=toronto_grouped2['Longitude']
                                  )


Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(2667, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Second Cup,43.802165,-79.196114,Coffee Shop
1,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
2,Malvern / Rouge,43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
3,Malvern / Rouge,43.806686,-79.194353,Lee Valley,43.803161,-79.199681,Hobby Shop
4,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course


In [28]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Alderwood / Long Branch,12,12,12,12,12,12
Bathurst Manor / Wilson Heights / Downsview North,21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,30,30,30,30,30,30
...,...,...,...,...,...,...
Willowdale,57,57,57,57,57,57
Woburn,5,5,5,5,5,5
Woodbine Heights,9,9,9,9,9,9


In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 297 uniques categories.


In [30]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2662,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2663,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2664,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2665,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_group = toronto_onehot.groupby(["Neighborhood"]).mean().reset_index()

print(toronto_group.shape)
toronto_group

(96, 297)


,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,Alderwood / Long Branch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.047619,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,Bedford Park / Lawrence Manor East,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Willowdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.017544,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
94,York Mills / Silver Hills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [32]:
toronto_group.shape

(96, 297)

In [33]:
num_top_venues = 5

for hood in toronto_group['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_group[toronto_group['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3                     Lounge  0.25
4                     Museum  0.00


----Alderwood / Long Branch----
               venue  freq
0        Pizza Place  0.17
1               Pool  0.08
2  Convenience Store  0.08
3           Pharmacy  0.08
4        Gas Station  0.08


----Bathurst Manor / Wilson Heights / Downsview North----
                       venue  freq
0                       Bank  0.10
1                Coffee Shop  0.10
2                Pizza Place  0.05
3                      Diner  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                     venue  freq
0                     Café  0.25
1      Japanese Restaurant  0.25
2       Chinese Restaurant  0.25
3                     Bank  0.25
4  New American Restaurant  0.00


----Bedford Park / Lawrence Manor East----
                venue  freq
0        

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_group['Neighborhood']

for ind in np.arange(toronto_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
1,Alderwood / Long Branch,Pizza Place,Gym,Convenience Store,Pharmacy,Skating Rink,Sandwich Place,Gas Station,Pub,Athletics & Sports,Pool
2,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Bridal Shop,Ice Cream Shop,Middle Eastern Restaurant,Sandwich Place,Deli / Bodega,Sushi Restaurant,Supermarket,Gas Station
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Discount Store
4,Bedford Park / Lawrence Manor East,Italian Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Sandwich Place,Restaurant,Pharmacy,Butcher,Café,Pub


# CLUSTER NEIGHBORHOODS

In [36]:
# set number of clusters
from sklearn.cluster import KMeans
kclusters = 4

toronto_grouped_clustering = toronto_group.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 3, 0, 0, 0, 0, 0, 0])

In [37]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', (kmeans.labels_))

toronto_merged = toronto_grouped2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



In [38]:
toronto_merged.dropna(inplace=True)
toronto_merged.head(20)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,0.0,Fast Food Restaurant,Coffee Shop,Hobby Shop,Dumpling Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,2.0,Golf Course,Bar,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Women's Store
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,0.0,Fried Chicken Joint,Thrift / Vintage Store,Breakfast Spot,Fast Food Restaurant,Medical Center,Bus Line,Mexican Restaurant,Bank,Intersection,Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Business Service,Convenience Store,Korean Restaurant,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Bakery,Indian Restaurant,Flower Shop,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Gas Station,Chinese Restaurant,Athletics & Sports
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Playground,Middle Eastern Restaurant,Pizza Place,Caribbean Restaurant,Basketball Court,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029,0.0,Discount Store,Hobby Shop,Department Store,Bus Line,Metro Station,Light Rail Station,Hockey Arena,Train Station,Coffee Shop,Convenience Store
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577,0.0,Intersection,Bakery,Diner,Metro Station,Bus Line,Soccer Field,Park,Convenience Store,Coffee Shop,Ice Cream Shop
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476,0.0,Wings Joint,Chinese Restaurant,Motel,American Restaurant,Hardware Store,Farm,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848,0.0,Café,General Entertainment,Skating Rink,Diner,College Stadium,Concert Hall,Discount Store,Farm,Falafel Restaurant,Event Space


In [47]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0.0,Fast Food Restaurant,Coffee Shop,Hobby Shop,Dumpling Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,Scarborough,0.0,Fried Chicken Joint,Thrift / Vintage Store,Breakfast Spot,Fast Food Restaurant,Medical Center,Bus Line,Mexican Restaurant,Bank,Intersection,Restaurant
3,Scarborough,0.0,Coffee Shop,Business Service,Convenience Store,Korean Restaurant,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
4,Scarborough,0.0,Bakery,Indian Restaurant,Flower Shop,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Gas Station,Chinese Restaurant,Athletics & Sports
5,Scarborough,0.0,Playground,Middle Eastern Restaurant,Pizza Place,Caribbean Restaurant,Basketball Court,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
92,Etobicoke,0.0,Gym,Thrift / Vintage Store,Burger Joint,Burrito Place,Kids Store,Sandwich Place,Supplement Shop,Bank,Bakery,Discount Store
95,Etobicoke,0.0,Pizza Place,Café,Pet Store,Liquor Store,Beer Store,Coffee Shop,Convenience Store,Cosmetics Shop,Electronics Store,Drugstore
96,North York,0.0,Pizza Place,Arts & Crafts Store,Empanada Restaurant,Bakery,Women's Store,Drugstore,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
97,North York,0.0,Food Service,Convenience Store,Baseball Field,Women's Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,1.0,Park,Cafeteria,Women's Store,Drugstore,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
23,North York,1.0,Park,Convenience Store,Bank,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,Central Toronto,1.0,Bus Line,Park,Swim School,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
48,Central Toronto,1.0,Park,Playground,Gym,Dumpling Restaurant,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
50,Downtown Toronto,1.0,Park,Playground,Trail,Donut Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
98,York,1.0,Park,Grocery Store,Convenience Store,Women's Store,Drugstore,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2.0,Golf Course,Bar,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Women's Store


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,3.0,Pharmacy,Pizza Place,Gas Station,Fast Food Restaurant,Bus Stop,Sandwich Place,Bank,Italian Restaurant,Intersection,Thai Restaurant
14,Scarborough,3.0,Chinese Restaurant,Fast Food Restaurant,BBQ Joint,Park,Drugstore,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
15,Scarborough,3.0,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Pharmacy,Bank,Pizza Place,Breakfast Spot,Shopping Mall,Coffee Shop,Supermarket
19,North York,3.0,Café,Bank,Chinese Restaurant,Japanese Restaurant,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Discount Store
35,East York,3.0,Pizza Place,Fast Food Restaurant,Gastropub,Rock Climbing Spot,Café,Bus Line,Breakfast Spot,Pharmacy,Bank,Intersection
80,York,3.0,Sandwich Place,Discount Store,Skating Rink,Recreation Center,Women's Store,Donut Shop,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
81,York,3.0,Pizza Place,Furniture / Home Store,Grocery Store,Park,Video Game Store,Indian Restaurant,Thai Restaurant,Bank,Gas Station,Dog Run
89,Etobicoke,3.0,Pizza Place,Gym,Convenience Store,Pharmacy,Skating Rink,Sandwich Place,Gas Station,Pub,Athletics & Sports,Pool
93,Etobicoke,3.0,Pharmacy,Shopping Mall,Grocery Store,Bank,Café,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant
94,Etobicoke,3.0,Pizza Place,Tea Room,Sandwich Place,Chinese Restaurant,Doner Restaurant,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop


# Observations

#### Cluster 1 has huge number of food places which is mostly fast food or Asian Food this leads to an expectation that huge number of Asian people are present in that area compared to others

#### Cluster 2 has huge number of Parks and Playgrounds

#### Cluster 3 is very small with most number of Golf courses and Bars and is expected to be a place with higher average age of the people living in that area 

#### Cluster 4 has a large number of Restaurants and Shopping areas with a lot of utility stores